In [1]:
from models.rnn.rnn_plain import RNN_Plain
from models.rnn.rnn_attention import RNN_Attention

from lang import load_data_int_seq
from utils import accuracy_score

%load_ext autoreload
%autoreload 2

In [2]:
output_lang, input_lang, train, X_test, y_test = load_data_int_seq()

## Plain RNN

In [3]:
algo = RNN_Plain(symbols = "+*-0123456789t", 
output_sequence_length = 9, 
encoded_seq_length = 9, 
num_epochs = 10000, 
input_size = input_lang.n_words, 
hidden_size = 256, 
output_size=output_lang.n_words, 
embedding_size = 28, 
batch_size = 32, 
num_gru_layers = 2,
dropout_prob = 0.3,
calc_magnitude_on=False)


EncoderRNN(
  (embedding): Embedding(2564, 28)
  (gru): GRU(28, 256, num_layers=2, dropout=0.3)
)
DecoderRNN(
  (embedding): Embedding(16, 28)
  (gru): GRU(28, 256, num_layers=2, dropout=0.3)
  (out): Linear(in_features=256, out_features=16, bias=True)
  (softmax): LogSoftmax(dim=1)
)


In [4]:
algo.train(input_lang, output_lang, train)


[0.48836109 0.48836109 0.72113929 0.73344058 0.69625284 0.
 0.02791446 0.02081756 0.0317941  0.03188872 0.02053369 0.02781983
 0.03028009 0.03690386 0.02952309 1.        ]
1m 16s (- 11m 24s) (1000 10%) 1.0452
2m 21s (- 9m 27s) (2000 20%) 0.5965
3m 26s (- 8m 1s) (3000 30%) 0.5948
4m 27s (- 6m 40s) (4000 40%) 0.5943
5m 29s (- 5m 29s) (5000 50%) 0.5919
6m 34s (- 4m 23s) (6000 60%) 0.5910
7m 36s (- 3m 15s) (7000 70%) 0.5893
8m 34s (- 2m 8s) (8000 80%) 0.5906
9m 39s (- 1m 4s) (9000 90%) 0.5901
10m 41s (- 0m 0s) (10000 100%) 0.5885


In [5]:
pred = algo.infer(input_lang, output_lang, X_test[:1000])
pred[:10]

['t*t*t*t*t',
 't*t*t*t*t',
 't*t*t*t*t',
 't*t*t*t*t',
 't*t*t*t*t',
 't*t*t*t*t',
 't*t*t*t*t',
 't*t*t*t*t',
 't*t*t*t*t',
 't*t*t*t*t']

In [6]:
accuracy_score(pred, y_test[:1000])

0.0

### Plain RNN with Magnitude loss

In [ ]:
magn_algo = RNN_Plain(symbols = "+*-0123456789t", output_sequence_length = 9, encoded_seq_length = 9, num_epochs= 5000, input_size=input_lang.n_words, hidden_size=512, output_size=output_lang.n_words, calc_magnitude_on=True)
magn_algo.train(input_lang, output_lang, train)

In [ ]:
pred = magn_algo.infer(input_lang, output_lang, X_test[:1000])
pred[:25]

In [ ]:
accuracy_score(pred, y_test[:1000])

##  Attention-based

In [ ]:
algo_attn = RNN_Attention(symbols = "+*-0123456789t", output_sequence_length = 9, encoded_seq_length = 9, num_epochs= 5000, input_size=input_lang.n_words, hidden_size=512, output_size=output_lang.n_words, calc_magnitude_on=False)
algo_attn.train(input_lang, output_lang, train)

In [ ]:
pred = algo_attn.infer(input_lang, output_lang, X_test[:1000])
pred[:25]

In [ ]:
accuracy_score(pred, y_test[:1000])

### Attention with magnitude loss

In [ ]:
algo_attn_magn = RNN_Attention(symbols = "+*-0123456789t", output_sequence_length = 9, encoded_seq_length = 9, num_epochs= 5000, input_size=input_lang.n_words, hidden_size=512, output_size=output_lang.n_words, calc_magnitude_on=True)
algo_attn_magn.train(input_lang, output_lang, train)

In [ ]:
pred = algo_attn_magn.infer(input_lang, output_lang, X_test[:1000])
pred[:25]

In [ ]:
accuracy_score(pred, y_test[:1000])

# Helper Stuff

In [ ]:
import torch
import torch.nn as nn
m = nn.LogSoftmax()
input = torch.randn(2, 3)
print(input)
output = m(input)
print(output)

In [ ]:
# 2D loss example (used, for example, with image inputs)
N, C = 5, 4
loss = nn.NLLLoss()

# input is of size N x C x height x width
data = torch.randn(N, 16, 10, 10)
conv = nn.Conv2d(16, C, (3, 3))
m = nn.LogSoftmax(dim=1)

# print(data.shape)
activated = m(conv(data))
print("output: ", activated.shape)

# each element in target has to have 0 <= value < C
target = torch.empty(N, 8, 8, dtype=torch.long).random_(0, C)
print("target: ", target.shape)
output = loss(activated, target)

# output.backward()

In [ ]:
m = nn.LogSoftmax(dim=1)
loss = nn.NLLLoss()
# input is of size N x C = 3 x 5
# input = minibatchsize, number of categories

input = torch.randn(3, 5, requires_grad=True)
# print(input.shape)
activation = m(input)
print(activation.shape)
# each element in target has to have 0 <= value < C
target = torch.tensor([1, 0, 4])
print(target.shape)
output = loss(activation, target)
# output.backward()

In [ ]:
from line_profiler import LineProfiler

stuff_to_evaluate = [1,2,3]
def function_to_evaluate(stuff_to_evaluate):
    pass

lp = LineProfiler()
lp_wrapper = lp(function_to_evaluate)
lp_wrapper(stuff_to_evaluate)
lp.print_stats()